In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn


import sklearn
from sklearn import preprocessing
from datetime import datetime
import random
import math
import holidays
us_holidays = holidays.UnitedStates()

## Config

In [5]:
## change the data_path to where you store data
data_path = 'Data'

## Preprocess Weather Data (Handle missing values)

In [6]:
df_w = pd.read_csv( data_path + '/weather_train.csv')

df_w2 = df_w.copy(deep=True)

df_w2['air_temperature'] = df_w2['air_temperature'].interpolate(method='quadratic', limit_direction='both')
df_w2['air_temperature'] = df_w2['air_temperature'].interpolate(method='nearest', limit_direction='both')

df_w2['dew_temperature'] = df_w2['dew_temperature'].interpolate(method='linear', limit_direction='both')
df_w2['dew_temperature'] = df_w2['dew_temperature'].interpolate(method='nearest', limit_direction='both')

df_w2['sea_level_pressure'] = df_w2['sea_level_pressure'].interpolate(method='quadratic', limit_direction='both')
df_w2['sea_level_pressure'] = df_w2['sea_level_pressure'].interpolate(method='nearest', limit_direction='both')
df_w2['sea_level_pressure'] = np.where((df_w2['site_id']==5), np.nan, df_w2['sea_level_pressure'])

df_w2['wind_speed'] = df_w2['wind_speed'].interpolate(method='linear', limit_direction='both')
df_w2['wind_speed'] = df_w2['wind_speed'].interpolate(method='nearest', limit_direction='both')


df_w2['wind_direction cat'] = np.where(df_w2['wind_direction'].isna(), 'Unknown',
                                  np.where(df_w2['wind_speed']==0, 'No Wind',
                                  np.where((df_w2['wind_direction']<22.5)  | (df_w2['wind_direction']>=337.5), 'East',
                                  np.where((df_w2['wind_direction']<67.5)  & (df_w2['wind_direction']>=22.5), 'Northeast',
                                  np.where((df_w2['wind_direction']<112.5) & (df_w2['wind_direction']>=67.5), 'North',
                                  np.where((df_w2['wind_direction']<157.5) & (df_w2['wind_direction']>=112.5), 'Northwest',
                                  np.where((df_w2['wind_direction']<202.5) & (df_w2['wind_direction']>=157.5), 'West',
                                  np.where((df_w2['wind_direction']<247.5) & (df_w2['wind_direction']>=202.5), 'Southwest',
                                  np.where((df_w2['wind_direction']<292.5) & (df_w2['wind_direction']>=247.5), 'South', 
                                  'Southeast')))))))))

df_w2['wind_direction cat2'] = np.where(df_w2['wind_direction'].isna(), 'Unknown',
                                  np.where(df_w2['wind_speed']==0, 'No Wind',
                                  np.where((df_w2['wind_direction']<45)  & (df_w2['wind_direction']>=0), 'East',
                                  np.where((df_w2['wind_direction']<90)  & (df_w2['wind_direction']>=45), 'Northeast',
                                  np.where((df_w2['wind_direction']<135) & (df_w2['wind_direction']>=90), 'North',
                                  np.where((df_w2['wind_direction']<180) & (df_w2['wind_direction']>=135), 'Northwest',
                                  np.where((df_w2['wind_direction']<225) & (df_w2['wind_direction']>=180), 'West',
                                  np.where((df_w2['wind_direction']<270) & (df_w2['wind_direction']>=225), 'Southwest',
                                  np.where((df_w2['wind_direction']<315) & (df_w2['wind_direction']>=270), 'South', 
                                  'Southeast')))))))))


df_w2['precipitation'] = np.where(df_w2['precip_depth_1_hr'].isna(), 'Unknown', 
                                  np.where(df_w2['precip_depth_1_hr']<=0, 'N', 'Y'))

df_w2['precipitation sign'] = np.where(df_w2['precip_depth_1_hr'].isna(), 'Unknown',
                                   np.where(df_w2['precip_depth_1_hr']<0, 'Negative',
                                   np.where(df_w2['precip_depth_1_hr']==0, 'Zero', 'Positive')))

# df_w2['wind_speed_log'] = np.log(df_w2['wind_speed']+1)


In [7]:
#####
# weather shift
shift_hour = 1
######

cols = ['air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'wind_direction cat',
       'wind_direction cat2', 'precipitation', 'precipitation sign']

for col in cols:
    df_w2[col+'_lag'] = df_w2[col].shift(shift_hour)
    df_w2[col+'_lag'] = df_w2[col].shift(shift_hour)


In [8]:
df_w2.to_csv( data_path + '/weather_train_processed_lag.csv', index=False)
del df_w2

## Save as feather files for faster processing and saving memory

In [9]:
train_df = pd.read_csv( data_path + '/train.csv')
weather_train_df = pd.read_csv(data_path + '/weather_train_processed_lag.csv')
building_meta_df = pd.read_csv(data_path + '/building_metadata.csv')

train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])

train_df.to_feather( data_path + '/train.feather')
weather_train_df.to_feather( data_path + '/weather_train_processed_lag.feather')
building_meta_df.to_feather( data_path + '/building_metadata.feather')

del train_df
del weather_train_df
del building_meta_df

## Merge data

In [10]:
train_df = pd.read_feather( data_path + '/train.feather')
weather_train_df = pd.read_feather( data_path + '/weather_train_processed_lag.feather')
building_meta_df = pd.read_feather( data_path + '/building_metadata.feather')

In [11]:
train_df = train_df.merge(building_meta_df, left_on = "building_id", right_on = "building_id", how = "left")
train_df = train_df.merge(weather_train_df, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
del weather_train_df
del building_meta_df

In [12]:
train_df.shape

(20216100, 31)

## Add New Features

In [13]:
%%time
le = preprocessing.LabelEncoder()
train_df['primary_use'] = le.fit_transform(train_df['primary_use'])

move = ['meter_reading', 'timestamp'] + [x for x in train_df.columns if x not in ['meter_reading', 'timestamp']]
train_df = train_df[move]

mask =(train_df['site_id']==0) & (train_df['timestamp'] < '2016-06-01')
train_df = train_df[~mask]

# this may take mins
train_df['date_month'] = train_df['timestamp'].map(lambda x: x.month)
train_df['date_day'] = train_df['timestamp'].map(lambda x: x.day)
train_df['iso_week'] = train_df['timestamp'].map(lambda x: x.isocalendar()[1])
train_df['iso_weekend'] = train_df['timestamp'].map(lambda x: x.isocalendar()[2])
train_df['hour'] = train_df['timestamp'].map(lambda x: x.hour)
train_df['holidays'] = train_df['timestamp'].map(lambda x: 1 if x in us_holidays else 0)

Wall time: 3min 58s


In [14]:
use = ['meter_reading', 'building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'wind_direction cat', 'wind_direction cat2',
        'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction cat_lag', 'wind_direction cat2_lag',
        'date_month', 'date_day',
       'iso_week', 'iso_weekend', 'hour', 'holidays']

train_df = train_df[use]

train_df['wind_direction cat'][train_df['wind_direction cat'].isna()] = 'Unknown'
train_df['wind_direction cat2'][train_df['wind_direction cat2'].isna()] = 'Unknown'
train_df['wind_direction cat_lag'][train_df['wind_direction cat_lag'].isna()] = 'Unknown'
train_df['wind_direction cat2_lag'][train_df['wind_direction cat2_lag'].isna()] = 'Unknown'

cat_need_encode = ['wind_direction cat', 'wind_direction cat2',
                  'wind_direction cat_lag', 'wind_direction cat2_lag', 
                  ]

for var in cat_need_encode:
    print(var)
    le = preprocessing.LabelEncoder()
    train_df[var] = le.fit_transform(train_df[var])

wind_direction cat
wind_direction cat2
wind_direction cat_lag
wind_direction cat2_lag


## Reduce CSV file size
Since AutoMLs can not take feather file, we prepared a set of CSV files for feeding AutoML platforms.

In [15]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [16]:
train_df = reduce_mem_usage(train_df)

Mem. usage decreased to 1189.02 Mb (74.2% reduction)


## Rename the columns to prevent errors on the platforms

In [17]:
use = ['meter_reading', 'building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'wind_direction_cat', 'wind_direction_cat2',
        'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction_cat_lag', 'wind_direction_cat2_lag',
        'date_month', 'date_day',
       'iso_week', 'iso_weekend', 'hour', 'holidays']

train_df.columns = use

In [18]:
%%time
train_df = train_df.reset_index(drop=True)
train_df.to_csv( data_path + 'processed_automl_google.csv', index=False)

Wall time: 6min 46s
